In [35]:
#imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_log_error, make_scorer, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, MinMaxScaler, PolynomialFeatures
from common.CommonFunctions import plot_learning_curve, train_and_eval, my_score
import lightgbm as lgb
import mmap
import shutil
import contextlib

In [36]:

originalData = pd.read_csv('data/train.csv', index_col='id')
ratings = []
with open('data/rating.tsv', 'rt') as f:
    # memory-map the file, size 0 means whole file
    with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as m:
        for code in originalData.imdb_id:
            #mmap returns everything in byte
            word = bytes(code, 'utf-8')
            i = m.find(word)   # search for last occurrence of 'word'
            #i = np.maximum(i-20,0)
            m.readline()
            if i > -1:
                m.seek(i)             # seek to the location
                line = m.readline().decode("utf-8")   # read to the end of the line
                s = line.split('\t')
                #[code[1] = imdb_id, code[2] = rating, [code[3] = numVotes
                ratings.append([s[0],s[1],s[2]])                
            else :
                print("Not found", word)
            m.seek(0)

In [37]:
array = np.asarray(ratings)
newFeatures = pd.DataFrame({'imdb_id':array[:,0], 'rating':array[:,1], 'numRatings':array[:,2]})
newFeatures.numRatings = newFeatures.numRatings.str.replace("\n", "").astype(int)
newFeatures.to_csv('imdb.csv', index=False)

In [38]:
df = pd.read_csv('data/train.csv', index_col="id")


In [39]:
df_imdb = df.merge(newFeatures, on=['imdb_id'], how='left')

In [41]:
df = pd.read_csv('exported_features/all_features.csv')
df['rating'] = df_imdb['rating']
df['numRatings'] = df_imdb['numRatings']
df.to_csv('exported_features/all_features_imdb.csv')

In [43]:
df.head()

,id,revenue,budgetlog,normalizedBudgetLog,popularitylog,budgetPop,runtime,Age,hasStar,NumStar,...,History,Drama,Science Fiction,Romance,Fantasy,Horror,Animation,numberGenres,rating,numRatings
0,1,12314651,16.454568,0.355784,2.024905,33.318941,93.0,2,1,2,...,False,False,False,False,False,False,False,1.0,5.1,34245
1,2,95149435,17.504390,0.781577,2.224504,38.938587,113.0,13,0,0,...,False,True,False,True,False,False,False,4.0,5.8,67100
2,3,13092000,15.009433,0.118750,3.044522,45.696557,105.0,3,1,1,...,False,True,False,False,False,False,False,1.0,8.5,606130
3,4,16000000,13.997833,0.039112,1.429099,20.004289,122.0,5,0,0,...,False,True,False,False,False,False,False,2.0,8.1,51442
4,5,3923970,15.894952,0.275906,0.764570,12.152800,118.0,8,0,0,...,False,False,False,False,False,False,False,2.0,5.8,284
